In [64]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import learning_curve, validation_curve


def plot_learning_curve(
    estimator,
    X,
    y,
    title="Learning Curve",
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
    scoring=None,
):
    """
    This is a custom modification of the code present here:
    https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
    
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))

    scoring: string, callable or None, optional, default: None
        A string (see model evaluation documentation) or a scorer callable object / function
        with signature scorer(estimator, X, y).
    """

    fig, axes = plt.subplots(1, 1, figsize=(10, 5))

    axes.set_title(title)
    if ylim is not None:
        axes.set_ylim(*ylim)
    axes.set_xlabel("Training examples")
    axes.set_ylabel("Score")

    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=scoring
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axes.grid(True)
    axes.fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes.fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes.plot(train_sizes, train_scores_mean, "o-", color="r", label="Training score")
    axes.plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes.legend(loc="best")
    return fig, axes


def plot_validation_curve(
    estimator,
    X,
    y,
    ylim=None,
    cv=None,
    n_jobs=None,
    param_name=None,
    param_range=None,
    scoring=None,
):
    """
    referred from :
    https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics

    :param estimator: object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.
    :param X: array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.
    :param y: array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.
    :param ylim: tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.
    :param cv: int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.
    :param n_jobs: int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.
    :param param_range: array-like, shape (n_values,)
    The values of the parameter that will be evaluated.
    :param param_name: string
    Name of the parameter that will be varied.
    :param scoring: string, callable or None, optional, default: None
        A string (see model evaluation documentation) or a scorer callable object / function
        with signature scorer(estimator, X, y).
    :return: fig


    """
    train_scores, test_scores = validation_curve(
        estimator,
        X,
        y,
        param_name=param_name,
        param_range=param_range,
        scoring=scoring,
        n_jobs=n_jobs,
        cv=cv,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    fig, axes = plt.subplots(1, 1, figsize=(10, 5))
    axes.grid(True)
    axes.set_title(f"Validation Curve with {estimator.__class__}")
    axes.set_xlabel(f"{param_name}")
    axes.set_ylabel(f"{scoring}")
    if ylim is not None:
        axes.set_ylim(*ylim)

    lw = 2
    axes.plot(
        param_range,
        train_scores_mean,
        label="Training score",
        color="darkorange",
        lw=lw,
    )
    axes.fill_between(
        param_range,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.2,
        color="darkorange",
        lw=lw,
    )
    axes.plot(
        param_range,
        test_scores_mean,
        label="Cross-validation score",
        color="navy",
        lw=lw,
    )
    axes.fill_between(
        param_range,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.2,
        color="navy",
        lw=lw,
    )
    axes.legend(loc="best")
    return fig, axes


In [66]:
import pandas as pd
import seaborn as sns
sns.set()

from pathlib import Path

# classifiers we will use
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost

#imputers
from sklearn.impute import SimpleImputer, KNNImputer

# model selection bits
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, ParameterGrid, ParameterSampler
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit, GroupShuffleSplit, GroupKFold, StratifiedKFold
from sklearn.model_selection import learning_curve, validation_curve

# evaluation
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

import scipy

import openpyxl

Let's load all the data. 

In [5]:
# Load the competition datasets into Pandas DataFrame
path = Path("/Users/13392/Documents/amp-parkinsons-disease-progression-prediction")
proteins = pd.read_csv(path/"train_proteins.csv")
peptides = pd.read_csv(path/"train_peptides.csv")
clinical = pd.read_csv(path/"train_clinical_data.csv")
supplemental = pd.read_csv(path/"supplemental_clinical_data.csv")

As discussed previously, we are dropping the entire "medication status" column, because:
1) Over 50% values are NaN. 
2) the test dataset will not have this data. 

In [6]:
# drop the "medication status" column (due to over 50% NaN values), keep a copy of the original
# for later access. 
clinical_copy = clinical.copy()

clinical.drop('upd23b_clinical_state_on_medication', axis=1, inplace=True)

In [7]:

targets = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']
ids = ['patient_id', 'visit_id']
month = ['visit_month']

Let's see how much remains of 'NaN'

In [8]:
print(f'NaN value count:\n{clinical.isna().sum()}')
clinical

NaN value count:
visit_id          0
patient_id        0
visit_month       0
updrs_1           1
updrs_2           2
updrs_3          25
updrs_4        1038
dtype: int64


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,55_0,55,0,10.0,6.0,15.0,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0
4,55_12,55,12,10.0,10.0,41.0,0.0
...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0
2611,65043_54,65043,54,4.0,8.0,11.0,1.0
2612,65043_60,65043,60,6.0,6.0,16.0,1.0
2613,65043_72,65043,72,3.0,9.0,14.0,1.0


Significant, but manageable with some kind of imputation. Let's count the number of visits each patient has on record. 

In [9]:
cols = ['patient_id', 'num_entries']
patient_list = clinical.patient_id.unique()
n_list = []
p_list = []

for patient in patient_list:
    n=len(clinical[clinical.patient_id==patient].index)

    n_list.append(n)
    p_list.append(patient)

df_visits_by_patient = pd.DataFrame(list(zip(p_list, n_list)), columns=cols)

df_visits_by_patient


,patient_id,num_entries
0,55,13
1,942,15
2,1517,10
3,1923,7
4,2660,6
...,...,...
243,63875,9
244,63889,10
245,64669,15
246,64674,16


NaN values in the "Proteins"  and "Peptides" datasets. 

In [10]:
print(f'NaN value count:\n{proteins.isna().sum()}')
proteins

NaN value count:
visit_id       0
visit_month    0
patient_id     0
UniProt        0
NPX            0
dtype: int64


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [11]:
print(f'NaN value count:\n{peptides.isna().sum()}')

peptides

NaN value count:
visit_id            0
visit_month         0
patient_id          0
UniProt             0
Peptide             0
PeptideAbundance    0
dtype: int64


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


Great! No NaN values at all in them. Let's find out how many of each patient's visits have protein/peptide data. 

In [12]:
cols = ['patient_id', 'num_entries_protein']
patient_list = proteins.patient_id.unique()
n_list = []
p_list = []

for patient in patient_list:
    n=len(proteins[proteins.patient_id==patient].visit_id.unique())

    n_list.append(n)
    p_list.append(patient)

df_recorded_visits_protein = pd.DataFrame(list(zip(p_list, n_list)), columns=cols)

df_recorded_visits_protein


,patient_id,num_entries_protein
0,55,4
1,1517,4
2,1923,3
3,2660,5
4,3636,3
...,...,...
243,52998,3
244,54979,3
245,58597,3
246,7508,3


In [13]:
cols = ['patient_id', 'num_entries_peptide']
patient_list = peptides.patient_id.unique()
n_list = []
p_list = []

for patient in patient_list:
    n=len(peptides[peptides.patient_id==patient].visit_id.unique())

    n_list.append(n)
    p_list.append(patient)

df_recorded_visits_peptide = pd.DataFrame(list(zip(p_list, n_list)), columns=cols)

df_recorded_visits_peptide

,patient_id,num_entries_peptide
0,55,4
1,1517,4
2,1923,3
3,2660,5
4,3636,3
...,...,...
243,52998,3
244,54979,3
245,58597,3
246,7508,3


In [14]:
df = pd.merge(df_recorded_visits_protein, df_recorded_visits_peptide, on='patient_id', how='left')
df = pd.merge(df_visits_by_patient, df, on='patient_id', how='left')
df.head(10)

,patient_id,num_entries,num_entries_protein,num_entries_peptide
0,55,13,4,4
1,942,15,4,4
2,1517,10,4,4
3,1923,7,3,3
4,2660,6,5,5
5,3636,14,3,3
6,3863,9,5,5
7,4161,12,6,6
8,4172,8,7,7
9,4923,11,5,5


It's not looking great - we can clearly see that, for most of the patients, only 1/2 to 1/3 of the visits contain protein and peptide records.

Something to think about for later on...

Now we will "pivot" the datasets so the unique coding for each protein/peptide becomes a feature for the models to learn on. 

In [15]:
df_proteins = proteins.pivot(index=['patient_id', 'visit_month', 'visit_id'], columns='UniProt', values='NPX').rename_axis(columns=None).reset_index()

df_peptides = peptides.pivot(index=['patient_id', 'visit_month', 'visit_id'], columns='Peptide', values='PeptideAbundance').rename_axis(columns=None).reset_index()


In [16]:
df_proteins

,patient_id,visit_month,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,55,0,55_0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,...,365475.0,35528.00,97005.6,23122.5,60912.6,408698.0,NaN,29758.8,23833.7,18953.5
1,55,6,55_6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,...,405676.0,30332.60,109174.0,23499.8,51655.8,369870.0,NaN,22935.2,17722.5,16642.7
2,55,12,55_12,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,...,303953.0,43026.20,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9
3,55,36,55_36,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,...,303597.0,48188.40,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
4,942,6,942_6,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,...,253373.0,27431.80,93796.7,17450.9,21299.1,306621.0,82335.5,24018.7,18939.5,15251.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,64674_84,NaN,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,...,260021.0,7139.93,104277.0,10500.0,21944.2,136725.0,62217.5,NaN,10287.7,13848.2
1109,65043,0,65043_0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,...,186414.0,25897.80,NaN,21480.7,57364.0,416142.0,37584.6,NaN,28346.5,35617.5
1110,65043,12,65043_12,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,...,301343.0,22343.40,105626.0,20500.8,54011.2,380072.0,40588.9,NaN,17035.7,37064.2
1111,65043,24,65043_24,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,...,300439.0,52143.60,139291.0,19449.2,66569.9,300948.0,36150.4,NaN,21286.3,39587.9


In [17]:
df_peptides

,patient_id,visit_month,visit_id,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55,0,55_0,8984260.0,53855.6,8579740.0,NaN,19735.4,114400.0,46371.1,...,201158.0,16492.30,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55,6,55_6,8279770.0,45251.9,8655890.0,49927.5,23820.4,90539.4,38652.4,...,171079.0,13198.80,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,55,12,55_12,8382390.0,53000.9,8995640.0,45519.2,17813.5,147312.0,45840.9,...,231772.0,17873.80,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,55,36,55_36,10671500.0,58108.4,9985420.0,52374.0,19373.3,64356.1,49793.2,...,185290.0,18580.50,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,942,6,942_6,6177730.0,42682.6,3596660.0,25698.8,17130.6,86471.5,41007.9,...,226314.0,6399.80,NaN,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,64674_84,7083630.0,35656.1,6273100.0,NaN,NaN,15479.2,NaN,...,203523.0,3835.58,4901220.0,40325.9,335625.0,49250.4,64076.3,667993.0,38472.5,21949.1
1109,65043,0,65043_0,7818630.0,95033.0,5119260.0,57483.7,11610.0,270739.0,42527.3,...,257361.0,18316.60,2514660.0,51444.6,530245.0,156148.0,157548.0,336625.0,48423.2,10915.8
1110,65043,12,65043_12,8070390.0,76532.7,8233520.0,54260.6,11631.9,230169.0,42255.5,...,230437.0,16703.20,2481560.0,44405.0,543391.0,159828.0,161207.0,330337.0,45368.1,19023.2
1111,65043,24,65043_24,7608150.0,75401.6,9168030.0,NaN,13313.9,220202.0,46914.1,...,251228.0,18326.20,2939460.0,50588.2,597869.0,148032.0,192857.0,388125.0,65101.0,20790.1


Because not all 1000+ proteins and peptides are measured at each recorded visit, some NaN values should be now expected. 

In [18]:
df_proteins.isna().sum().sort_values(ascending=False)

Q99829        624
Q99832        507
Q562R1        497
P01780        459
Q6UX71        452
             ... 
P02766          0
P02765          0
P02751          0
P02749          0
patient_id      0
Length: 230, dtype: int64

In [19]:
df_peptides.isna().sum().sort_values(ascending=False)

QALPQVR                   624
EPQVYTLPPSRDELTK          550
TPSGLYLGTC(UniMod_4)ER    523
SLEDQVEMLR                514
HYEGSTVPEK                508
                         ... 
visit_id                    0
IPTTFENGR                   0
AIGYLNTGYQR                 0
NILTSNNIDVK                 0
patient_id                  0
Length: 971, dtype: int64

We are going to combine the protein and peptide data, and check once again the status of NaN values. They should remain unchanged because we haven't done anything with them.

In [20]:
prot_pept_df = pd.merge(df_proteins, df_peptides, on=['patient_id','visit_month','visit_id'], how='left')
prot_pept_df

,patient_id,visit_month,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55,0,55_0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,...,201158.0,16492.30,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55,6,55_6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,...,171079.0,13198.80,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,55,12,55_12,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,...,231772.0,17873.80,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,55,36,55_36,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,...,185290.0,18580.50,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,942,6,942_6,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,...,226314.0,6399.80,NaN,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,64674_84,NaN,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,...,203523.0,3835.58,4901220.0,40325.9,335625.0,49250.4,64076.3,667993.0,38472.5,21949.1
1109,65043,0,65043_0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,...,257361.0,18316.60,2514660.0,51444.6,530245.0,156148.0,157548.0,336625.0,48423.2,10915.8
1110,65043,12,65043_12,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,...,230437.0,16703.20,2481560.0,44405.0,543391.0,159828.0,161207.0,330337.0,45368.1,19023.2
1111,65043,24,65043_24,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,...,251228.0,18326.20,2939460.0,50588.2,597869.0,148032.0,192857.0,388125.0,65101.0,20790.1


In [21]:
prot_pept_df.isna().sum().sort_values(ascending=False)

Q99829                    624
QALPQVR                   624
EPQVYTLPPSRDELTK          550
TPSGLYLGTC(UniMod_4)ER    523
SLEDQVEMLR                514
                         ... 
P41222                      0
P02774                      0
P02787                      0
P02790                      0
patient_id                  0
Length: 1198, dtype: int64

In [22]:
patient_list=prot_pept_df['patient_id'].unique()
patient_list

array([   55,   942,  1517,  1923,  2660,  3636,  3863,  4161,  4172,
        4923,  5027,  5036,  5178,  5645,  5742,  6054,  6211,  6420,
        7051,  7117,  7151,  7265,  7508,  7568,  7832,  7886,  8344,
        8699, 10053, 10138, 10174, 10541, 10715, 10718, 11459, 11686,
       11928, 12516, 12636, 12703, 12755, 12931, 13360, 13368, 13618,
       13804, 13852, 13968, 14035, 14124, 14242, 14270, 14344, 14450,
       14811, 15009, 15245, 15504, 15590, 16238, 16347, 16566, 16574,
       16778, 16931, 17154, 17201, 17414, 17727, 18183, 18204, 18553,
       18560, 19088, 20212, 20216, 20352, 20404, 20460, 20581, 20664,
       20707, 20791, 20792, 21126, 21537, 21729, 22126, 22623, 23175,
       23192, 23244, 23391, 23636, 24278, 24690, 24818, 24820, 24911,
       25562, 25739, 25750, 25827, 25911, 26005, 26104, 26210, 26809,
       27079, 27300, 27464, 27468, 27607, 27715, 27872, 27893, 27971,
       27987, 28327, 28342, 28818, 29313, 29417, 30119, 30155, 30416,
       30894, 30951,

Here there are two ways to impute the data:
1) combine "clinical" and "prot_pept_df" first then impute, or
2) first impute them separately, then combine. 

We'll do both and see how the outcomes differ.

I am choosing to use sklearn's KNN imputer because, with a bit of clever coding, I can use all the availble data for a given patient to impute missing values. 

In [23]:
# Method 1: Combine first, impute next. 
big_data = pd.merge(clinical, prot_pept_df, on=['patient_id','visit_month','visit_id'], how='left')
big_data.head(30)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,NaN,11254.30,732430.0,39585.8,...,201158.0,16492.30,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,13163.60,630465.0,35220.8,...,171079.0,13198.80,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
3,55_9,55,9,8.0,9.0,30.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,15257.60,815083.0,41650.9,...,231772.0,17873.80,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
5,55_18,55,18,7.0,13.0,38.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55_24,55,24,16.0,9.0,49.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,55_30,55,30,14.0,13.0,49.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,55_36,55,36,17.0,18.0,51.0,0.0,13530.80,753832.0,43048.9,...,185290.0,18580.50,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
9,55_42,55,42,12.0,20.0,41.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#imputing
big_data_patient_list = big_data.patient_id.unique()
data_imputed_list = []
for patient_id in big_data_patient_list:
    masked_data = big_data[big_data['patient_id']==patient_id]
    num_rows = len(masked_data.index)
    knn = KNNImputer(missing_values=np.nan, keep_empty_features=True, n_neighbors=num_rows)
    X_knn = knn.fit_transform(masked_data)
    X_knn_df = pd.DataFrame(X_knn, columns = big_data.columns)
    data_imputed_list.append(X_knn_df)

big_data_imputed = pd.concat(data_imputed_list)

In [25]:
big_data_imputed.head(10)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,550.0,55.0,0.0,10.0,6.0,15.0,0.0,11254.300,732430.0,39585.8,...,201158.00,16492.30,3810270.0,106894.000,580667.0,131155.00,165851.00,437305.0,46289.200,14898.400
1,553.0,55.0,3.0,10.0,7.0,25.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
2,556.0,55.0,6.0,8.0,10.0,34.0,0.0,13163.600,630465.0,35220.8,...,171079.00,13198.80,4119520.0,113385.000,514861.0,103512.00,144607.00,457891.0,40047.700,20703.900
3,559.0,55.0,9.0,8.0,9.0,30.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
4,5512.0,55.0,12.0,10.0,10.0,41.0,0.0,15257.600,815083.0,41650.9,...,231772.00,17873.80,5474140.0,116286.000,711815.0,136943.00,181763.00,452253.0,54725.100,21841.100
5,5518.0,55.0,18.0,7.0,13.0,38.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
6,5524.0,55.0,24.0,16.0,9.0,49.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
7,5530.0,55.0,30.0,14.0,13.0,49.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
8,5536.0,55.0,36.0,17.0,18.0,51.0,0.0,13530.800,753832.0,43048.9,...,185290.00,18580.50,2659660.0,90936.900,679163.0,128593.00,203680.00,498621.0,52792.700,13973.700
9,5542.0,55.0,42.0,12.0,20.0,41.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275


In [26]:
#resetting index to match original.
big_data_imputed.reset_index(inplace=True, drop=True)
big_data_imputed.head(10)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,550.0,55.0,0.0,10.0,6.0,15.0,0.0,11254.300,732430.0,39585.8,...,201158.00,16492.30,3810270.0,106894.000,580667.0,131155.00,165851.00,437305.0,46289.200,14898.400
1,553.0,55.0,3.0,10.0,7.0,25.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
2,556.0,55.0,6.0,8.0,10.0,34.0,0.0,13163.600,630465.0,35220.8,...,171079.00,13198.80,4119520.0,113385.000,514861.0,103512.00,144607.00,457891.0,40047.700,20703.900
3,559.0,55.0,9.0,8.0,9.0,30.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
4,5512.0,55.0,12.0,10.0,10.0,41.0,0.0,15257.600,815083.0,41650.9,...,231772.00,17873.80,5474140.0,116286.000,711815.0,136943.00,181763.00,452253.0,54725.100,21841.100
5,5518.0,55.0,18.0,7.0,13.0,38.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
6,5524.0,55.0,24.0,16.0,9.0,49.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
7,5530.0,55.0,30.0,14.0,13.0,49.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275
8,5536.0,55.0,36.0,17.0,18.0,51.0,0.0,13530.800,753832.0,43048.9,...,185290.00,18580.50,2659660.0,90936.900,679163.0,128593.00,203680.00,498621.0,52792.700,13973.700
9,5542.0,55.0,42.0,12.0,20.0,41.0,0.0,13301.575,732952.5,39876.6,...,197324.75,16536.35,4015897.5,106875.475,621626.5,125050.75,173975.25,461517.5,48463.675,17854.275


In [27]:
#relabel patient id, visit id, and visit month.
ls = [ids, month]
for col in ls:
    big_data_imputed[col] = big_data[col]

In [28]:
big_data_imputed.head(30)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,0.000000,11254.300,732430.0,39585.8,...,201158.00,16492.3000,3810270.0,106894.000,580667.00,131155.000,165851.00,437305.00,46289.200,14898.400000
1,55_3,55,3,10.0,7.0,25.0,0.000000,13301.575,732952.5,39876.6,...,197324.75,16536.3500,4015897.5,106875.475,621626.50,125050.750,173975.25,461517.50,48463.675,17854.275000
2,55_6,55,6,8.0,10.0,34.0,0.000000,13163.600,630465.0,35220.8,...,171079.00,13198.8000,4119520.0,113385.000,514861.00,103512.000,144607.00,457891.00,40047.700,20703.900000
3,55_9,55,9,8.0,9.0,30.0,0.000000,13301.575,732952.5,39876.6,...,197324.75,16536.3500,4015897.5,106875.475,621626.50,125050.750,173975.25,461517.50,48463.675,17854.275000
4,55_12,55,12,10.0,10.0,41.0,0.000000,15257.600,815083.0,41650.9,...,231772.00,17873.8000,5474140.0,116286.000,711815.00,136943.000,181763.00,452253.00,54725.100,21841.100000
5,55_18,55,18,7.0,13.0,38.0,0.000000,13301.575,732952.5,39876.6,...,197324.75,16536.3500,4015897.5,106875.475,621626.50,125050.750,173975.25,461517.50,48463.675,17854.275000
6,55_24,55,24,16.0,9.0,49.0,0.000000,13301.575,732952.5,39876.6,...,197324.75,16536.3500,4015897.5,106875.475,621626.50,125050.750,173975.25,461517.50,48463.675,17854.275000
7,55_30,55,30,14.0,13.0,49.0,0.000000,13301.575,732952.5,39876.6,...,197324.75,16536.3500,4015897.5,106875.475,621626.50,125050.750,173975.25,461517.50,48463.675,17854.275000
8,55_36,55,36,17.0,18.0,51.0,0.000000,13530.800,753832.0,43048.9,...,185290.00,18580.5000,2659660.0,90936.900,679163.00,128593.000,203680.00,498621.00,52792.700,13973.700000
9,55_42,55,42,12.0,20.0,41.0,0.000000,13301.575,732952.5,39876.6,...,197324.75,16536.3500,4015897.5,106875.475,621626.50,125050.750,173975.25,461517.50,48463.675,17854.275000


Next is Method #2: impute each dataset first, then combine. 

Imputing prot_pept_df:

In [29]:
data_imputed_list = []
for patient_id in patient_list:
    masked_data = prot_pept_df[prot_pept_df['patient_id']==patient_id]
    num_rows = len(masked_data.index)
    knn = KNNImputer(missing_values=np.nan, keep_empty_features=True, n_neighbors=num_rows)
    X_knn = knn.fit_transform(masked_data)
    X_knn_df = pd.DataFrame(X_knn, columns = prot_pept_df.columns)
    data_imputed_list.append(X_knn_df)
prot_pept_imputed = pd.concat(data_imputed_list)

In [30]:
prot_pept_imputed.isna().sum()

patient_id               0
visit_month              0
visit_id                 0
O00391                   0
O00533                   0
                        ..
YVNKEIQNAVNGVK           0
YWGVASFLQK               0
YYC(UniMod_4)FQGNQFLR    0
YYTYLIMNK                0
YYWGGQYTWDMAK            0
Length: 1198, dtype: int64

In [31]:
prot_pept_imputed.reset_index(inplace=True, drop=True)
prot_pept_imputed

,patient_id,visit_month,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55.0,0.0,550.0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,...,201158.0,16492.30,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55.0,6.0,556.0,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,...,171079.0,13198.80,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,55.0,12.0,5512.0,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,...,231772.0,17873.80,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,55.0,36.0,5536.0,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,...,185290.0,18580.50,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,942.0,6.0,9426.0,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,...,226314.0,6399.80,374307.0,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674.0,84.0,6467484.0,0.0,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,...,203523.0,3835.58,4901220.0,40325.9,335625.0,49250.4,64076.3,667993.0,38472.5,21949.1
1109,65043.0,0.0,650430.0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,...,257361.0,18316.60,2514660.0,51444.6,530245.0,156148.0,157548.0,336625.0,48423.2,10915.8
1110,65043.0,12.0,6504312.0,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,...,230437.0,16703.20,2481560.0,44405.0,543391.0,159828.0,161207.0,330337.0,45368.1,19023.2
1111,65043.0,24.0,6504324.0,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,...,251228.0,18326.20,2939460.0,50588.2,597869.0,148032.0,192857.0,388125.0,65101.0,20790.1


In [32]:
prot_pept_imputed = prot_pept_imputed.astype({'patient_id': 'int', 'visit_month': 'int'})
prot_pept_imputed['visit_id'] = prot_pept_df['visit_id']
print(prot_pept_imputed.dtypes)

patient_id                 int32
visit_month                int32
visit_id                  object
O00391                   float64
O00533                   float64
                          ...   
YVNKEIQNAVNGVK           float64
YWGVASFLQK               float64
YYC(UniMod_4)FQGNQFLR    float64
YYTYLIMNK                float64
YYWGGQYTWDMAK            float64
Length: 1198, dtype: object


In [33]:
prot_pept_imputed

,patient_id,visit_month,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55,0,55_0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,...,201158.0,16492.30,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55,6,55_6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,...,171079.0,13198.80,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,55,12,55_12,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,...,231772.0,17873.80,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,55,36,55_36,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,...,185290.0,18580.50,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,942,6,942_6,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,...,226314.0,6399.80,374307.0,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,64674_84,0.0,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,...,203523.0,3835.58,4901220.0,40325.9,335625.0,49250.4,64076.3,667993.0,38472.5,21949.1
1109,65043,0,65043_0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,...,257361.0,18316.60,2514660.0,51444.6,530245.0,156148.0,157548.0,336625.0,48423.2,10915.8
1110,65043,12,65043_12,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,...,230437.0,16703.20,2481560.0,44405.0,543391.0,159828.0,161207.0,330337.0,45368.1,19023.2
1111,65043,24,65043_24,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,...,251228.0,18326.20,2939460.0,50588.2,597869.0,148032.0,192857.0,388125.0,65101.0,20790.1


In [34]:
print(clinical.isna().sum())
print(f'dataset shape: {clinical.shape}')

visit_id          0
patient_id        0
visit_month       0
updrs_1           1
updrs_2           2
updrs_3          25
updrs_4        1038
dtype: int64
dataset shape: (2615, 7)


In [35]:
data_imputed_list = []
for patient_id in patient_list:
    masked_data = clinical[clinical['patient_id']==patient_id]
    num_rows = len(masked_data.index)
    knn = KNNImputer(missing_values=np.nan, keep_empty_features=True, n_neighbors=num_rows)
    X_knn = knn.fit_transform(masked_data)
    X_knn_df = pd.DataFrame(X_knn, columns = clinical.columns)
    data_imputed_list.append(X_knn_df)
clinical_imputed = pd.concat(data_imputed_list)

In [36]:
print(clinical_imputed.isna().sum())
print(f'imputed clinical dataset shape: {clinical_imputed.shape}')

visit_id       0
patient_id     0
visit_month    0
updrs_1        0
updrs_2        0
updrs_3        0
updrs_4        0
dtype: int64
imputed clinical dataset shape: (2615, 7)


In [37]:
clinical_imputed.reset_index(inplace=True, drop=True)
clinical_imputed

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,550.0,55.0,0.0,10.0,6.0,15.0,0.0
1,553.0,55.0,3.0,10.0,7.0,25.0,0.0
2,556.0,55.0,6.0,8.0,10.0,34.0,0.0
3,559.0,55.0,9.0,8.0,9.0,30.0,0.0
4,5512.0,55.0,12.0,10.0,10.0,41.0,0.0
...,...,...,...,...,...,...,...
2610,6504348.0,65043.0,48.0,7.0,6.0,13.0,0.0
2611,6504354.0,65043.0,54.0,4.0,8.0,11.0,1.0
2612,6504360.0,65043.0,60.0,6.0,6.0,16.0,1.0
2613,6504372.0,65043.0,72.0,3.0,9.0,14.0,1.0


In [38]:
clinical_imputed = clinical_imputed.astype({'patient_id': 'int', 'visit_month': 'int'})
clinical_imputed['visit_id'] = clinical['visit_id']
print(clinical_imputed.dtypes)

visit_id        object
patient_id       int32
visit_month      int32
updrs_1        float64
updrs_2        float64
updrs_3        float64
updrs_4        float64
dtype: object


In [39]:

prot_pept_clinical = pd.merge(prot_pept_imputed, clinical_imputed, on=['visit_id', 'visit_month', 'patient_id'], how='left')


In [40]:
print(prot_pept_clinical.isna().sum())
print(f'merged protein peptide and clinical dataset shape: {prot_pept_clinical.shape}')

patient_id        0
visit_month       0
visit_id          0
O00391            0
O00533            0
                 ..
YYWGGQYTWDMAK     0
updrs_1          45
updrs_2          45
updrs_3          45
updrs_4          45
Length: 1202, dtype: int64
merged protein peptide and clinical dataset shape: (1113, 1202)


Interestingly, with Method #2 (first impute separately, then combine) we end up with some additional NaN values that need to be dealt with. 

In [41]:
df = prot_pept_clinical[prot_pept_clinical.columns[prot_pept_clinical.isna().any()]]

df1 = df[df.isna().any(axis=1)]

print(df1)
print()
print(f'Remaining NaN values is a dataframe of: {df1.shape}')

      updrs_1  updrs_2  updrs_3  updrs_4
16        NaN      NaN      NaN      NaN
47        NaN      NaN      NaN      NaN
50        NaN      NaN      NaN      NaN
57        NaN      NaN      NaN      NaN
92        NaN      NaN      NaN      NaN
95        NaN      NaN      NaN      NaN
156       NaN      NaN      NaN      NaN
172       NaN      NaN      NaN      NaN
182       NaN      NaN      NaN      NaN
216       NaN      NaN      NaN      NaN
219       NaN      NaN      NaN      NaN
239       NaN      NaN      NaN      NaN
269       NaN      NaN      NaN      NaN
338       NaN      NaN      NaN      NaN
342       NaN      NaN      NaN      NaN
364       NaN      NaN      NaN      NaN
378       NaN      NaN      NaN      NaN
405       NaN      NaN      NaN      NaN
406       NaN      NaN      NaN      NaN
411       NaN      NaN      NaN      NaN
431       NaN      NaN      NaN      NaN
439       NaN      NaN      NaN      NaN
474       NaN      NaN      NaN      NaN
499       NaN   

In this case, all four parts of the UPDRS score are empty. The best course of action would be to drop the 45 rows entirely, since there is no reasonable way to impute the NaN values (or else we wouldn't need to train a model to predict these values).

In [42]:
prot_pept_clinical.dropna(subset = targets, inplace=True, axis=0)
prot_pept_clinical.isna().sum()

patient_id       0
visit_month      0
visit_id         0
O00391           0
O00533           0
                ..
YYWGGQYTWDMAK    0
updrs_1          0
updrs_2          0
updrs_3          0
updrs_4          0
Length: 1202, dtype: int64

Ok, so now we have two imputed combined datasets to work with. Before introducing ML models, I'll first define a function to calculate SMAPE scores. 

In [43]:
def smape_score(actual, predicted):
    sum = 0
    for a, p in zip(actual, predicted):
        if a==0 and p==0:
            pass
        else:
            sum += (np.abs(p-a))/(np.abs(p)+np.abs(a))*2
    return sum/len(actual)*100

The models to be tested include:

Random Forest Regressor, HistGradientBoostingRegressor, Tree based AdaBoostRegressor, Tree based XGBRegressor.


Before specifying parameters for each model, my first question is: if I don't specify any parameters, what does the model do on its own? 

Let's first take a look at RandomForestRegressor training on big_data_imputed. 

In [ ]:
RFR = RandomForestRegressor() # not specifying any parameters.

X = big_data_imputed.drop(columns=targets, axis=1)

for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    #fitting and testing
    RFR.fit(X_train,y_train)    
    y_predict = RFR.predict(X_test)
    #scoring
    mae = mean_absolute_error(y_test, y_predict)
    smape = smape_score(y_test, y_predict)
    #saving the scores in respective lists
    print(f'MAE: {mae}; SMAPE: {smape}')
    print()

    max_depth = list()

    for tree in RFR.estimators_:
        max_depth.append(tree.tree_.max_depth)

    print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
    print(f"max depth {max(max_depth)}")
    print(f"min depth {min(max_depth)}")
    print()

In [ ]:
depth, counts = np.unique(max_depth, return_counts=True)
print(f'unique depths: {depth}')
print(f'occurances: {counts}')

A random forest regressor with no specified parameters, training on this dataset, reaches a maximum depth of 32 in 318 (out of 500) trees, which means the models appears to favor deep trees - an important feature to keep in mind. 

For RandomForestRegressor, argurably the most important parameters to control are:

n_estimators, the number of decision trees in the forest;

max_depth, the max number of splits that each tree is allowed.

I'll select some parameters, based on feedback from the previous uncontrolled RandomForest Regressor, which noticeably favors deeper trees for this dataset. So I will set a range of max_depth from 8 to 60, and n_estimators (number of trees in the forest) from 100 (default) to 800. I'll then compare these models to the default model.

In [44]:

RFR = RandomForestRegressor()

params = {'n_estimators': [100,200,500,800],
                 'max_depth': [5,10,20,30,40,50]}

random_params = list(ParameterSampler(params, n_iter=5, random_state=1))

random_params


[{'n_estimators': 200, 'max_depth': 30},
 {'n_estimators': 500, 'max_depth': 40},
 {'n_estimators': 800, 'max_depth': 5},
 {'n_estimators': 500, 'max_depth': 30},
 {'n_estimators': 100, 'max_depth': 50}]

I'll first save these pseudo randomly generated parameters in a dataframe, for later referencing.

In [55]:
params_df = pd.DataFrame(random_params)
params_df

,n_estimators,max_depth
0,200,30
1,500,40
2,800,5
3,500,30
4,100,50


I'll first test the RandomForestRegressor model on "big_data_imputed", i.e. data that was first combined then imputed. 



In [65]:
#randomforestregressor 
smape_list_RFR1 = []
mae_list_RFR1 = []
actual_depth_RFR1 = []

#defining features
X = big_data_imputed.drop(columns=targets, axis=1)

for target in targets:
    #setting target variable
    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    for params in random_params:
        #fitting and testing
        RFR.set_params(**params)
        RFR.fit(X_train,y_train)    
        y_predict = RFR.predict(X_test)
        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        #saving the scpres in respective lists
        mae_list_RFR1.append(mae)
        smape_list_RFR1.append(smape)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')
        #still good to know how deep the trees are actually going.
        max_depth = list()
        for tree in RFR.estimators_:
            max_depth.append(tree.tree_.max_depth)
        print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
        print(f"max depth {max(max_depth)}")
        print(f"min depth {min(max_depth)}")
        print()
        actual_depth_RFR1.append(max(max_depth))
    print()


updrs_1: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.3710152223897705; SMAPE: 49.079224752172
avg max depth 27.2
max depth 30
min depth 20

Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.362293129770993; SMAPE: 48.95117769888684
avg max depth 27.4
max depth 40
min depth 22

Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 3.193118093613285; SMAPE: 56.67153959452127
avg max depth 5.0
max depth 5
min depth 5

Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.3889340353697426; SMAPE: 48.87420306665155
avg max depth 27.3
max depth 30
min depth 22

Parameter: {'n_estimators': 100, 'max_depth': 50}; MAE: 2.370603053435114; SMAPE: 48.790888022796125
avg max depth 27.7
max depth 36
min depth 22


updrs_2: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.2929478371501273; SMAPE: 67.61782424287573
avg max depth 24.7
max depth 30
min depth 20

Paramet

I will store the scores and their respective parameters in a dataframe, for easy referencing. 

In [68]:
RFR_df = pd.concat([params_df,params_df,params_df,params_df], ignore_index=True)
RFR_df['Actual Depth Reached'] = actual_depth_RFR1
RFR_df['MAE'] = mae_list_RFR1
RFR_df['SMAPE'] = smape_list_RFR1
RFR_df['UPDRS'] = ""
RFR_df.loc[:4, 'UPDRS']=1
RFR_df.loc[5:9, 'UPDRS']=2
RFR_df.loc[10:14, 'UPDRS']=3
RFR_df.loc[15:, 'UPDRS']=4

RFR_df

,n_estimators,max_depth,Actual Depth Reached,MAE,SMAPE,UPDRS
0,200,30,30,2.371015,49.079225,1
1,500,40,40,2.362293,48.951178,1
2,800,5,5,3.193118,56.671540,1
3,500,30,30,2.388934,48.874203,1
4,100,50,36,2.370603,48.790888,1
5,200,30,30,2.292948,67.617824,2
6,500,40,35,2.295665,66.986994,2
7,800,5,5,3.510394,77.739767,2
8,500,30,30,2.317915,68.757632,2
9,100,50,30,2.305992,66.808430,2


In [ ]:
RFR_df.to_excel('initial_model_selection_results.xlsx', sheet_name='RFR') # saving the results in an excel file.

Note that the lowest MAE scores may not correspond with the lowest SMAPE scores, but I have already discussed and established that SMAPE may not be the best metric for this model. Mean Absolute Error, on the other hand, paints a clearer picture.

Also note that the training dataset contains 2353 samples, and the testing dataset contains 262 samples. 

Now we'll test the same set of parameters on the same model, on "prot_pept_clinical" data, i.e. data that was first independently imputed, then combined. Like I have previously pointed out, this method produces a dataset that it less than 50% the size of Method 1. I would expect the trained model to perform more poorly. 

In [69]:
#defining features
X = prot_pept_clinical.drop(columns=targets, axis=1)
smape_list_RFR2 = []
mae_list_RFR2 = []
actual_depth_RFR2 = []

for target in targets:
    #setting target variable
    y = prot_pept_clinical[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    for params in random_params:
        #fitting and testing
        RFR.set_params(**params)
        RFR.fit(X_train,y_train)    
        y_predict = RFR.predict(X_test)
        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        #saving the scpres in respective lists
        mae_list_RFR2.append(mae)
        smape_list_RFR2.append(smape)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')
        #still good to know how deep the trees are actually going.
        max_depth = list()
        for tree in RFR.estimators_:
            max_depth.append(tree.tree_.max_depth)
        print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
        print(f"max depth {max(max_depth)}")
        print(f"min depth {min(max_depth)}")
        print()
        actual_depth_RFR2.append(max(max_depth))

    print()

updrs_1: 961 samples in training, 107 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.9116187904515014; SMAPE: 60.20244121517351
avg max depth 23.0
max depth 30
min depth 16

Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.8869906542056074; SMAPE: 59.89508030344424
avg max depth 23.1
max depth 32
min depth 17

Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 2.942980385306648; SMAPE: 60.54869502661885
avg max depth 5.0
max depth 5
min depth 5

Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.8899603619696843; SMAPE: 59.83601681099554
avg max depth 22.8
max depth 30
min depth 16

Parameter: {'n_estimators': 100, 'max_depth': 50}; MAE: 2.95411214953271; SMAPE: 60.88069746729068
avg max depth 22.5
max depth 30
min depth 17


updrs_2: 961 samples in training, 107 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 3.157273012195177; SMAPE: 83.65750406348964
avg max depth 22.6
max depth 30
min depth 16

Parameter

In [71]:
RFR_df_2 = pd.concat([params_df,params_df,params_df,params_df], ignore_index=True)
#RFR_df_2['Actual Depth Reached'] = actual_depth_RFR2
RFR_df_2['MAE'] = mae_list_RFR2
RFR_df_2['SMAPE'] = smape_list_RFR2
RFR_df_2['UPDRS'] = ""
RFR_df_2.loc[:4, 'UPDRS']=1
RFR_df_2.loc[5:9, 'UPDRS']=2
RFR_df_2.loc[10:14, 'UPDRS']=3
RFR_df_2.loc[15:, 'UPDRS']=4

RFR_df_2

,n_estimators,max_depth,MAE,SMAPE,UPDRS
0,200,30,2.911619,60.202441,1
1,500,40,2.886991,59.895080,1
2,800,5,2.942980,60.548695,1
3,500,30,2.889960,59.836017,1
4,100,50,2.954112,60.880697,1
5,200,30,3.157273,83.657504,2
6,500,40,3.145121,83.317725,2
7,800,5,3.459868,86.309901,2
8,500,30,3.169432,83.788541,2
9,100,50,3.229252,84.224771,2


Overall, the model performs better across all tested parameters with the first dataset ("big_data_imputed"). This is as expected, since it contains more than 2 times the data as this second dataset. We will forgo the initial random parameter search with this smaller dataset on all the other models for now.

In [ ]:
#patient_id stratefied splitting
'''
smape_list = []
mae_list = []

for target in targets:
    X = big_data_imputed.drop(columns=targets, axis=1)
    y = big_data_imputed[target]

    for i, (train_index, test_index) in enumerate(gss.split(X, y, groups=X.patient_id)):
    
        X_train = X.loc[train_index]
        y_train = y.loc[train_index]
        X_test = X.loc[test_index]
        y_test = y.loc[test_index]
        
        print(f'{target} Split {i+1}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
        RFR.fit(X_train, y_train)

        y_predict = RFR.predict(X_test)

        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)

        smape_list.append(smape)
        mae_list.append(mae)

        print(f'MAE score for {target}: {mae}')
        print(f'SMAPE score for {target}: {smape}')
        print()

    mae_average = np.mean(mae_list)
    smape_average = np.mean(smape_list)

    print(f'MAE average score: {mae_average}')
    print(f'SMAPE average score: {smape_average}')
    print()

        

'''


I'll first test the Histogram Gradient Boosting Regressor (HGBR) without specifying parameters, and see where the dataset takes the model. 
Note: n_estimators (number of trees) is labeled as "max_iter" for HGBR. 

Note: unfortunately there is no easy way to extract information like the exact depth reached by each tree, like I have previously done with RandomForestRegressor. So I'll use the same randomly generated parameters previously.

In [79]:
hgb = HistGradientBoostingRegressor()
print(hgb.get_params())
X = big_data_imputed.drop(columns=targets, axis=1)


for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=10)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')

    #Randomly generate a dictionary containing 10 combinations of parameters. 
    
    #fitting and testing
    hgb.fit(X_train,y_train)    
    y_predict = hgb.predict(X_test)
    #scoring
    mae = mean_absolute_error(y_test, y_predict)
    smape = smape_score(y_test, y_predict)
    #saving scores in their lists
    print(f'MAE: {mae}; SMAPE: {smape}')
    #finding actual tested parameters
    print(f'Number of iterations: {hgb.n_iter_}')
    print(f'number of trees per iteration: {hgb.n_trees_per_iteration_} ')
    print()

{'categorical_features': None, 'early_stopping': 'auto', 'interaction_cst': None, 'l2_regularization': 0.0, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_bins': 255, 'max_depth': None, 'max_iter': 100, 'max_leaf_nodes': 31, 'min_samples_leaf': 20, 'monotonic_cst': None, 'n_iter_no_change': 10, 'quantile': None, 'random_state': None, 'scoring': 'loss', 'tol': 1e-07, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
updrs_1: 2353 samples in training, 262 samples in testing.
MAE: 2.413906042706124; SMAPE: 52.37888245935983
Number of iterations: 100
number of trees per iteration: 1 

updrs_2: 2353 samples in training, 262 samples in testing.
MAE: 2.2997074085595823; SMAPE: 69.2213090591029
Number of iterations: 100
number of trees per iteration: 1 

updrs_3: 2353 samples in training, 262 samples in testing.
MAE: 5.971028618733929; SMAPE: 56.31828494260663
Number of iterations: 100
number of trees per iteration: 1 

updrs_4: 2353 samples in training, 262 samples in testin

In [104]:
params_HGBR = {'max_iter': [100,200,500,800],
                 'max_depth': [5,10,20,30,40,50]}
random_params_HGBR = list(ParameterSampler(params_HGBR, n_iter=5, random_state=1))

#save them in a dataframe
params_df_hgb = pd.DataFrame(random_params_HGBR)
params_df_hgb

,max_iter,max_depth
0,200,30
1,500,40
2,800,5
3,500,30
4,100,50


In [105]:
#HistGradientBoostingRegressor with big_data_imputed

hgb = HistGradientBoostingRegressor()

X = big_data_imputed.drop(columns=targets, axis=1)

smape_list_hgb = []
mae_list_hgb = []

for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')

    #Randomly generate a dictionary containing 10 combinations of parameters. 
    
    for params in random_params_HGBR:
        #fitting and testing
        hgb.set_params(**params)
        hgb.fit(X_train,y_train)    
        y_predict = hgb.predict(X_test)
        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        #saving scores in their lists
        mae_list_hgb.append(mae)
        smape_list_hgb.append(smape)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')

    print()

updrs_1: 2353 samples in training, 262 samples in testing.
Parameter: {'max_iter': 200, 'max_depth': 30}; MAE: 2.334842439309835; SMAPE: 50.727589301539076
Parameter: {'max_iter': 500, 'max_depth': 40}; MAE: 2.3677531408160064; SMAPE: 51.92911172637387
Parameter: {'max_iter': 800, 'max_depth': 5}; MAE: 2.372718575757603; SMAPE: 50.967508370831396
Parameter: {'max_iter': 500, 'max_depth': 30}; MAE: 2.3677531408160064; SMAPE: 51.92911172637387
Parameter: {'max_iter': 100, 'max_depth': 50}; MAE: 2.344057988015238; SMAPE: 49.832565058737025

updrs_2: 2353 samples in training, 262 samples in testing.
Parameter: {'max_iter': 200, 'max_depth': 30}; MAE: 2.2161996709655534; SMAPE: 69.36927903008544
Parameter: {'max_iter': 500, 'max_depth': 40}; MAE: 2.2501852973382466; SMAPE: 69.66457485704936
Parameter: {'max_iter': 800, 'max_depth': 5}; MAE: 2.2584718354095203; SMAPE: 70.43037301977213
Parameter: {'max_iter': 500, 'max_depth': 30}; MAE: 2.2501852973382466; SMAPE: 69.66457485704936
Parameter:

Like I've done previously, I will combine the parameters and their scores into a single dataframe for easy referencing.

In [106]:
HGBR_df = pd.concat([params_df_hgb,params_df_hgb,params_df_hgb,params_df_hgb], ignore_index=True)
HGBR_df['MAE'] = mae_list_hgb
HGBR_df['SMAPE'] = smape_list_hgb
HGBR_df['UPDRS'] = ""
HGBR_df.loc[:4, 'UPDRS']=1
HGBR_df.loc[5:9, 'UPDRS']=2
HGBR_df.loc[10:14, 'UPDRS']=3
HGBR_df.loc[15:, 'UPDRS']=4
HGBR_df

,max_iter,max_depth,MAE,SMAPE,UPDRS
0,200,30,2.334842,50.727589,1
1,500,40,2.367753,51.929112,1
2,800,5,2.372719,50.967508,1
3,500,30,2.367753,51.929112,1
4,100,50,2.344058,49.832565,1
5,200,30,2.216200,69.369279,2
6,500,40,2.250185,69.664575,2
7,800,5,2.258472,70.430373,2
8,500,30,2.250185,69.664575,2
9,100,50,2.252021,69.703308,2


In [ ]:
HGBR_df.to_excel('initial_model_selection_results.xlsx', sheet_name='HGBR')

Now onto AdaBoost Regressor with Decision Tree base regressor.

In [108]:
#ADABOOST REGRESSOR with decision tree regressor
X = big_data_imputed.drop(columns=targets, axis=1)

smape_list_ada = []
mae_list_ada = []

for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    

    #fitting and testing
    ada = AdaBoostRegressor(DecisionTreeRegressor())
    print(ada.get_params())
    ada.fit(X_train,y_train)    
    y_predict = ada.predict(X_test)

    #scoring
    mae = mean_absolute_error(y_test, y_predict)
    smape = smape_score(y_test, y_predict)
    print(f'MAE: {mae}; SMAPE: {smape}')

    #saving scores in their lists
    mae_list_ada.append(mae)
    smape_list_ada.append(smape)

    max_depth = list()
    for tree in ada.estimators_:
        max_depth.append(tree.tree_.max_depth)
    print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
    print(f"max depth {max(max_depth)}")
    print(f"min depth {min(max_depth)}")
    print()

    print()

updrs_1: 2353 samples in training, 262 samples in testing.
{'base_estimator': 'deprecated', 'estimator__ccp_alpha': 0.0, 'estimator__criterion': 'squared_error', 'estimator__max_depth': None, 'estimator__max_features': None, 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__random_state': None, 'estimator__splitter': 'best', 'estimator': DecisionTreeRegressor(), 'learning_rate': 1.0, 'loss': 'linear', 'n_estimators': 50, 'random_state': None}
MAE: 2.4618320610687023; SMAPE: 51.81485976308136
avg max depth 25.8
max depth 35
min depth 21


updrs_2: 2353 samples in training, 262 samples in testing.
{'base_estimator': 'deprecated', 'estimator__ccp_alpha': 0.0, 'estimator__criterion': 'squared_error', 'estimator__max_depth': None, 'estimator__max_features': None, 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'est

We will again use the same parameters as previous models. 

In [2]:
random_params

NameError: name 'random_params' is not defined

In [110]:
#ADABOOST REGRESSOR with decision tree regressor
X = big_data_imputed.drop(columns=targets, axis=1)

smape_list_ada = []
mae_list_ada = []
actual_depth_ada = []


for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    for params in random_params:
        #fitting and testing
        dtr = DecisionTreeRegressor(max_depth=params['max_depth'])
        ada = AdaBoostRegressor(dtr, n_estimators=params['n_estimators'])
        ada.fit(X_train,y_train)    
        y_predict = ada.predict(X_test)

        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')

        #saving scores in their lists
        mae_list_ada.append(mae)
        smape_list_ada.append(smape)

        max_depth = list()
        for tree in ada.estimators_:
            max_depth.append(tree.tree_.max_depth)
        print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
        print(f"max depth {max(max_depth)}")
        print(f"min depth {min(max_depth)}")
        print()
        actual_depth_ada.append(max(max_depth))
    print()

updrs_1: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.4618320610687023; SMAPE: 51.249560705743704
avg max depth 25.1
max depth 30
min depth 19

Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.464885496183206; SMAPE: 50.20199874383017
avg max depth 25.3
max depth 36
min depth 20

Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 3.0019066881657497; SMAPE: 54.62420989345301
avg max depth 5.0
max depth 5
min depth 5

Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.4580152671755724; SMAPE: 50.66709293596736
avg max depth 25.3
max depth 30
min depth 19

Parameter: {'n_estimators': 100, 'max_depth': 50}; MAE: 2.5; SMAPE: 52.580875881365486
avg max depth 25.6
max depth 32
min depth 21


updrs_2: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.1603053435114505; SMAPE: 55.49326033690003
avg max depth 24.1
max depth 30
min depth 20

Parameter: {'n_es

In [111]:
ada_df = pd.concat([params_df,params_df,params_df,params_df], ignore_index=True)
ada_df['MAE'] = mae_list_ada
ada_df['SMAPE'] = smape_list_ada
ada_df['actual depth reached'] = actual_depth_ada
ada_df['UPDRS'] = ""
ada_df.loc[:4, 'UPDRS']=1
ada_df.loc[5:9, 'UPDRS']=2
ada_df.loc[10:14, 'UPDRS']=3
ada_df.loc[15:, 'UPDRS']=4

ada_df

,n_estimators,max_depth,MAE,SMAPE,actual depth reached,UPDRS
0,200,30,2.461832,51.249561,30,1
1,500,40,2.464885,50.201999,36,1
2,800,5,3.001907,54.624210,5,1
3,500,30,2.458015,50.667093,30,1
4,100,50,2.500000,52.580876,32,1
5,200,30,2.160305,55.493260,30,2
6,500,40,2.114504,54.043314,35,2
7,800,5,3.409691,77.171170,5,2
8,500,30,2.114504,53.553640,30,2
9,100,50,2.080153,53.136731,32,2


In [ ]:
ada_df.to_excel('initial_model_selection_results.xlsx', sheet_name='ADA')

In [45]:
xgbr = xgboost.XGBRegressor()
smape_list_xgb = []
mae_list_xgb = []

X = big_data_imputed.drop(columns=targets, axis=1)

for target in targets:

    y = big_data_imputed[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X.drop(columns='visit_id'), y, train_size=0.9, random_state=100) 

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    #fitting and testing
    xgbr.fit(X_train, y_train)
    y_predict = xgbr.predict(X_test)
    #print parameters
    print(f'xgbr.get_params(): {xgbr.get_params()}')
    print(f'xgbr.get_xgb_params(): {xgbr.get_xgb_params()}')
    #scoring
    mae = mean_absolute_error(y_test, y_predict)
    smape = smape_score(y_test, y_predict)
    #saving scores in their lists
    mae_list_xgb.append(mae)
    smape_list_xgb.append(smape)
    print(f'MAE: {mae}; SMAPE: {smape}')
    print()

updrs_1: 2353 samples in training, 262 samples in testing.
xgbr.get_params(): {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'predictor': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
xgbr.get_xgb_params(): {'ob

I will use the same set of "random_params".

In [51]:
random_params

[{'n_estimators': 200, 'max_depth': 30},
 {'n_estimators': 500, 'max_depth': 40},
 {'n_estimators': 800, 'max_depth': 5},
 {'n_estimators': 500, 'max_depth': 30},
 {'n_estimators': 100, 'max_depth': 50}]

In [53]:
#Tree based XGB Regressor using big_data_imputed

xgbr = xgboost.XGBRegressor()

smape_list_xgb = []
mae_list_xgb = []

X = big_data_imputed.drop(columns=targets, axis=1)

for target in targets:

    y = big_data_imputed[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X.drop(columns='visit_id'), y, train_size=0.9, random_state=100) 

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
   
    for params in random_params:
        #fitting and testing
        xgbr.set_params(**params)
        xgbr.fit(X_train, y_train)
        y_predict = xgbr.predict(X_test)
        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        #saving scores in their lists
        mae_list_xgb.append(mae)
        smape_list_xgb.append(smape)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')      
    print()

updrs_1: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.492170272096423; SMAPE: 50.378451071278576
Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.479126924984797; SMAPE: 50.195922608298225
Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 2.3878799083621334; SMAPE: 48.00519162482295
Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.4921702712433027; SMAPE: 50.378451071278576
Parameter: {'n_estimators': 100, 'max_depth': 50}; MAE: 2.478839653956469; SMAPE: 50.20406581388802

updrs_2: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.232429850859859; SMAPE: 66.78636772561987
Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.2293304183832743; SMAPE: 66.94807230418579
Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 2.2663253029659107; SMAPE: 65.09957269871339
Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.232429850859859; S

In [58]:
xgb_df = pd.concat([params_df,params_df,params_df,params_df], ignore_index=True)
xgb_df['MAE'] = mae_list_xgb
xgb_df['SMAPE'] = smape_list_xgb
xgb_df['UPDRS'] = ""
xgb_df.loc[:4, 'UPDRS']=1
xgb_df.loc[5:9, 'UPDRS']=2
xgb_df.loc[10:14, 'UPDRS']=3
xgb_df.loc[15:, 'UPDRS']=4

xgb_df

,n_estimators,max_depth,MAE,SMAPE,UPDRS
0,200,30,2.492170,50.378451,1
1,500,40,2.479127,50.195923,1
2,800,5,2.387880,48.005192,1
3,500,30,2.492170,50.378451,1
4,100,50,2.478840,50.204066,1
5,200,30,2.232430,66.786368,2
6,500,40,2.229330,66.948072,2
7,800,5,2.266325,65.099573,2
8,500,30,2.232430,66.786368,2
9,100,50,2.230404,66.931238,2


In [67]:
xgb_df.to_excel('initial_model_selection_results.xlsx', sheet_name='XGBR')